In [1]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens100KReader import Movielens100KReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens100KReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data() #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

DataSplitter_Holdout: Verifying data consistency...
DataSplitter_Holdout: Verifying data consistency... Passed!
DataSplitter_Holdout: DataReader: Movielens100K
	Num items: 1682
	Num users: 943
	Train 		quota 80.00 (80.00), 	interactions 80000, 	density 5.04E-02
	Validation 	quota 10.00 (10.00), 	interactions 10000, 	density 6.30E-03
	Test 		quota 10.00 (10.00), 	interactions 10000, 	density 6.30E-03





DataSplitter_Holdout: Done.
EvaluatorHoldout: Ignoring 27 ( 2.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 18 ( 1.9%) Users that have less than 1 test interactions


In [4]:
from Recommenders.NonPersonalizedRecommender import TopPop
top_pop_rec = TopPop(URM_train = URM_train)
top_pop_rec.fit()

TopPopRecommender: URM Detected 26 ( 1.5%) items with no interactions.


In [6]:
result_df, _ = evaluator_validation.evaluateRecommender(top_pop_rec)
result_df

EvaluatorHoldout: Processed 916 (100.0%) in 0.26 sec. Users per second: 3530


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.099454,0.14986,0.117416,0.04368,0.063093,0.259228,0.120767,0.107691,0.580786,0.33281,...,0.971368,0.564157,0.971368,0.011661,4.567562,0.952222,0.031427,0.470649,2.314344,0.095221
50,0.059083,0.287612,0.287119,0.01539,0.067632,0.270784,0.182321,0.098,0.808952,0.415335,...,0.971368,0.78579,0.971368,0.044878,6.530765,0.989437,0.120942,0.67294,1.680025,0.505584


In [8]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
item_knn_rec = ItemKNNCFRecommender(URM_train)
item_knn_rec.fit()
result_df, _ = evaluator_validation.evaluateRecommender(item_knn_rec)
result_df

ItemKNNCFRecommender: URM Detected 26 ( 1.5%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 1682 (100.0%), 14413.89 column/sec. Elapsed time 0.12 sec
EvaluatorHoldout: Processed 916 (100.0%) in 0.30 sec. Users per second: 3020


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.165611,0.258236,0.207101,0.086763,0.129613,0.400569,0.2141,0.184047,0.748908,0.579012,...,0.971368,0.727466,0.971368,0.089022,7.648795,0.995154,0.239908,0.788144,1.401286,0.105324
50,0.092205,0.512504,0.511855,0.030672,0.145648,0.410654,0.31614,0.156262,0.937773,0.708633,...,0.971368,0.910923,0.971368,0.174604,8.615639,0.998499,0.470546,0.887769,1.109345,0.552287


In [ ]:
[I 2024-02-22 15:05:56,224] Trial 11 finished with value: 0.13136624046802622 and parameters: {'batch_size': 128, 'embeddings_dim': 64, 'heads': 1, 'attention_blocks': 2, 'd_ff': 2048, 'epochs': 203, 'l2_reg': 1.2696354252027369e-05, 'learning_rate': 0.00037403191176238373, 'noise_timesteps': 1989, 'inference_timesteps': 1931, 'start_beta': 2.3795684515927568e-05, 'end_beta': 0.011802914069769507}. Best is trial 11 with value: 0.13136624046802622.


In [9]:
from Diffusion.DiffusionRecommender import MultiBlockAttentionDiffusionRecommender
diff_rec = MultiBlockAttentionDiffusionRecommender(URM_train=URM_train, use_gpu=True)
diff_rec.fit(    batch_size= 128, embeddings_dim= 64, heads= 1, attention_blocks= 2, d_ff= 2048, epochs= 203, l2_reg= 1.2696354252027369e-05, learning_rate= 0.00037403191176238373, noise_timesteps= 1989, inference_timesteps= 1931, start_beta= 2.3795684515927568e-05, end_beta= 0.011802914069769507)

MultiBlockAttentionDiffusionRecommender: URM Detected 26 ( 1.5%) items with no interactions.


/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Diffusion/DiffusionRecommender.py:648: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025541661/work/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  user_batch_tensor = torch.sparse_csr_tensor(user_batch_tensor.indptr,


MultiBlockAttentionDiffusionRecommender: Epoch 1, loss 8.73E+00
MultiBlockAttentionDiffusionRecommender: Epoch 1 of 203. Elapsed time 4.79 sec
MultiBlockAttentionDiffusionRecommender: Epoch 2, loss 8.26E+00
MultiBlockAttentionDiffusionRecommender: Epoch 2 of 203. Elapsed time 5.06 sec
MultiBlockAttentionDiffusionRecommender: Epoch 3, loss 8.05E+00
MultiBlockAttentionDiffusionRecommender: Epoch 3 of 203. Elapsed time 5.32 sec
MultiBlockAttentionDiffusionRecommender: Epoch 4, loss 7.80E+00
MultiBlockAttentionDiffusionRecommender: Epoch 4 of 203. Elapsed time 5.58 sec
MultiBlockAttentionDiffusionRecommender: Epoch 5, loss 7.77E+00
MultiBlockAttentionDiffusionRecommender: Epoch 5 of 203. Elapsed time 5.86 sec
MultiBlockAttentionDiffusionRecommender: Epoch 6, loss 7.73E+00
MultiBlockAttentionDiffusionRecommender: Epoch 6 of 203. Elapsed time 6.12 sec
MultiBlockAttentionDiffusionRecommender: Epoch 7, loss 8.01E+00
MultiBlockAttentionDiffusionRecommender: Epoch 7 of 203. Elapsed time 6.37 sec

In [10]:
resuld_df, _ = evaluator_validation.evaluateRecommender(diff_rec)
resuld_df

EvaluatorHoldout: Processed 916 (100.0%) in 15.09 sec. Users per second: 61


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.10393,0.157332,0.122995,0.047515,0.066075,0.26745,0.127969,0.112662,0.590611,0.351547,...,0.971368,0.573701,0.971368,0.017614,5.241384,0.968248,0.047469,0.540081,2.124048,0.096969
50,0.068624,0.374223,0.373693,0.017893,0.076439,0.282056,0.213063,0.115955,0.884279,0.452897,...,0.971368,0.858961,0.971368,0.061299,7.068104,0.992755,0.165196,0.728309,1.575948,0.513052


In [20]:
from Diffusion.DiffusionRecommender import MultiBlockAttentionDiffusionRecommender
diff_rec_low_time= MultiBlockAttentionDiffusionRecommender(URM_train=URM_train, use_gpu=True)
diff_rec_low_time.fit(batch_size = 128, embeddings_dim = 64, heads= 1, attention_blocks = 2, d_ff= 2048, epochs= 203, l2_reg= 1.2696354252027369e-05, learning_rate= 0.00037403191176238373, noise_timesteps= 300, inference_timesteps= 1, start_beta= 2.3795684515927568e-05, end_beta= 0.011802914069769507)

MultiBlockAttentionDiffusionRecommender: URM Detected 26 ( 1.5%) items with no interactions.
MultiBlockAttentionDiffusionRecommender: Epoch 1, loss 1.10E+01
MultiBlockAttentionDiffusionRecommender: Epoch 1 of 203. Elapsed time 0.43 sec
MultiBlockAttentionDiffusionRecommender: Epoch 2, loss 1.06E+01
MultiBlockAttentionDiffusionRecommender: Epoch 2 of 203. Elapsed time 0.73 sec
MultiBlockAttentionDiffusionRecommender: Epoch 3, loss 1.03E+01
MultiBlockAttentionDiffusionRecommender: Epoch 3 of 203. Elapsed time 0.99 sec
MultiBlockAttentionDiffusionRecommender: Epoch 4, loss 1.02E+01
MultiBlockAttentionDiffusionRecommender: Epoch 4 of 203. Elapsed time 1.26 sec
MultiBlockAttentionDiffusionRecommender: Epoch 5, loss 9.96E+00
MultiBlockAttentionDiffusionRecommender: Epoch 5 of 203. Elapsed time 1.55 sec
MultiBlockAttentionDiffusionRecommender: Epoch 6, loss 9.66E+00
MultiBlockAttentionDiffusionRecommender: Epoch 6 of 203. Elapsed time 1.81 sec
MultiBlockAttentionDiffusionRecommender: Epoch 7,

In [19]:
resuld_df, _ = evaluator_validation.evaluateRecommender(diff_rec_low_time)
resuld_df

EvaluatorHoldout: Processed 916 (100.0%) in 0.31 sec. Users per second: 2915


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.106114,0.164794,0.13224,0.047411,0.072631,0.280294,0.133918,0.117745,0.621179,0.362282,...,0.971368,0.603393,0.971368,0.02311,5.676303,0.975833,0.062279,0.584895,2.021342,0.097945
50,0.072031,0.401163,0.400623,0.0186,0.08879,0.294526,0.229099,0.122107,0.895197,0.471373,...,0.971368,0.869565,0.971368,0.076413,7.362477,0.994501,0.205927,0.758641,1.447615,0.522539
